In [1]:


import pandas as pd
import spacy

# Download spaCy English model if not already present
import spacy.cli
spacy.cli.download("en_core_web_sm")

# Load spaCy English model
nlp = spacy.load("en_core_web_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:

# Load dataset
df = pd.read_csv("carrefour datset.csv")
df.head()

,question,answer
0,Do you have fresh bananas in stock?,"Yes, fresh bananas are available daily in the ..."
1,How much is a kilogram of tomatoes?,A kilogram of tomatoes currently costs around ...
2,Do you sell local vegetables like sukuma wiki?,"Yes, we stock sukuma wiki and other local vege..."
3,Are your milk products sourced locally?,Most of our milk products are sourced from Ken...
4,What are the prices for cooking oil?,Prices for cooking oil range from KES 350 to K...


In [3]:

import nltk
import spacy
import string

# No need for stopwords if we're keeping them
# nltk.download('stopwords')  # <-- You can remove this

# Load spaCy's small English model
nlp = spacy.load("en_core_web_sm")

def preprocess_text_with_stopwords(text):
    # Lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize and lemmatize WITHOUT removing stopwords
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_punct and not token.is_space]
    return " ".join(tokens)

# Apply new preprocessing
df["cleaned_question"] = df["question"].apply(preprocess_text_with_stopwords)
df["cleaned_answer"] = df["answer"].apply(preprocess_text_with_stopwords)

# Preview result
df[["question", "cleaned_question"]].head()

,question,cleaned_question
0,Do you have fresh bananas in stock?,do you have fresh banana in stock
1,How much is a kilogram of tomatoes?,how much be a kilogram of tomato
2,Do you sell local vegetables like sukuma wiki?,do you sell local vegetable like sukuma wiki
3,Are your milk products sourced locally?,be your milk product source locally
4,What are the prices for cooking oil?,what be the price for cook oil


In [4]:
# Save the cleaned dataset to the current directory
df.to_csv("carrefour_cleaned_qa_dataset.csv", index=False)
df.head()

,question,answer,cleaned_question,cleaned_answer
0,Do you have fresh bananas in stock?,"Yes, fresh bananas are available daily in the ...",do you have fresh banana in stock,yes fresh banana be available daily in the fru...
1,How much is a kilogram of tomatoes?,A kilogram of tomatoes currently costs around ...,how much be a kilogram of tomato,a kilogram of tomato currently cost around kes...
2,Do you sell local vegetables like sukuma wiki?,"Yes, we stock sukuma wiki and other local vege...",do you sell local vegetable like sukuma wiki,yes we stock sukuma wiki and other local veget...
3,Are your milk products sourced locally?,Most of our milk products are sourced from Ken...,be your milk product source locally,most of our milk product be source from kenyan...
4,What are the prices for cooking oil?,Prices for cooking oil range from KES 350 to K...,what be the price for cook oil,price for cook oil range from kes 350 to kes 1...


In [5]:
df1 = df[["cleaned_question", "cleaned_answer"]]
df1.head()

,cleaned_question,cleaned_answer
0,do you have fresh banana in stock,yes fresh banana be available daily in the fru...
1,how much be a kilogram of tomato,a kilogram of tomato currently cost around kes...
2,do you sell local vegetable like sukuma wiki,yes we stock sukuma wiki and other local veget...
3,be your milk product source locally,most of our milk product be source from kenyan...
4,what be the price for cook oil,price for cook oil range from kes 350 to kes 1...


In [6]:
# Saving df1 to a CSV file
df1.to_csv("carrefour_cleaned_qa_dataset_v2.csv", index=False)


### **sentence_transformers**

In [ ]:
%pip install sentence-transformers

In [ ]:
# Import necessary libraries
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# Load cleaned questions and answers
df = pd.read_csv("carrefour_cleaned_qa_dataset_v2.csv")

# Load a lightweight BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode all questions
question_embeddings = model.encode(df['cleaned_question'].tolist(), convert_to_tensor=True)

def get_answer_bert(user_query):
    query_embedding = model.encode(user_query, convert_to_tensor=True)
    # Compute cosine similarities
    similarities = util.cos_sim(query_embedding, question_embeddings)[0]
    idx = similarities.argmax().item()
    return df.iloc[idx]['cleaned_answer']

# Example usage
print(get_answer_bert("What are your opening hours?"))

ModuleNotFoundError: No module named 'sentence_transformers'

### **Model Building using TF-IDF**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Load cleaned questions and answers
df = pd.read_csv("carrefour_cleaned_qa_dataset_v2.csv")

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['cleaned_question'])

def get_answer(user_query):
    user_query_clean = user_query.lower()
    user_vec = vectorizer.transform([user_query_clean])
    similarities = cosine_similarity(user_vec, tfidf_matrix)
    idx = similarities.argmax()
    return df.iloc[idx]['cleaned_answer']

# Example usage
print(get_answer("What are your opening hours?"))

most carrefour store operate from 800 am to 1000 pm daily


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load cleaned questions and answers
df = pd.read_csv("carrefour_cleaned_qa_dataset_v2.csv")

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['cleaned_question'])

def get_answer(user_query):
    user_query_clean = user_query.lower()
    user_vec = vectorizer.transform([user_query_clean])
    similarities = cosine_similarity(user_vec, tfidf_matrix)
    idx = similarities.argmax()
    return df.iloc[idx]['cleaned_answer']

# Interactive loop in notebook
while True:
    user_input = input("Ask a question (or type 'exit' to quit): ")
    if user_input.lower() == 'exit':
        break
    answer = get_answer(user_input)
    print("Answer:", answer)

Answer: yes fresh banana be available daily in the fruit section
Answer: yes all our meat be certify halal
Answer: yes all our meat be certify halal
Answer: yes mpesa be accept at all carrefour branch
Answer: yes fresh banana be available daily in the fruit section
